# DeepFake Detection - Fine-tuning with FaceForensics++
## EXP-004: FaceForensics++ 데이터셋으로 ViT 모델 파인튜닝

### ⚠️ 사전 준비:
1. **베이스라인 모델을 Drive에 업로드** (선택사항)
   - 로컬: `baseline/model/deep-fake-detector-v2-model/`
   - Drive: `/content/drive/MyDrive/deepfake_models/deep-fake-detector-v2-model/`
   - 업로드 안 하면 자동으로 원본 ViT 모델 사용

2. **FaceForensics++ 데이터 다운로드** (`download_faceforensics.ipynb` 실행)
   - Drive: `/content/drive/MyDrive/FaceForensics++/`

### 실행 순서:
1. 런타임 → 런타임 유형 변경 → **L4 GPU** 선택 ⭐
2. 셀 순서대로 실행
3. 최종 모델 다운로드 → `submit/models/`에 배치

### 예상 시간:
- 전처리 (얼굴 추출): 1-2시간
- 파인튜닝: 2-3시간
- **총 3-5시간**


---
## 1. 환경 설정


In [ ]:
# GPU 확인
!nvidia-smi


In [ ]:
# 필수 라이브러리 설치
%pip install -q transformers==4.30.0 torch torchvision pillow opencv-python dlib-bin tqdm scikit-learn


In [ ]:
import os
import json
import cv2
import dlib
import torch
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")


---
## 2. Google Drive 마운트


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# 작업 디렉토리 생성
WORK_DIR = "/content/deepfake_finetune"
DATA_DIR = f"{WORK_DIR}/data"
OUTPUT_DIR = f"{WORK_DIR}/output"
DRIVE_SAVE_DIR = "/content/drive/MyDrive/deepfake_models"

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DRIVE_SAVE_DIR, exist_ok=True)

print(f"Working Directory: {WORK_DIR}")
print(f"Model Save Directory: {DRIVE_SAVE_DIR}")


---
## 3. 데이터 준비

📦 **FaceForensics++ 사용 (추천)**
- `download_faceforensics.ipynb`로 다운로드 완료했다면
- Drive 경로만 확인하면 됨!


In [ ]:
# FaceForensics++ Drive 경로 확인
FF_DATA_PATH = "/content/drive/MyDrive/FaceForensics++"

if os.path.exists(FF_DATA_PATH):
    print("✅ FaceForensics++ 데이터 발견!")
    print(f"   경로: {FF_DATA_PATH}")
    
    # 데이터 구조 확인
    !ls -lh {FF_DATA_PATH}
else:
    print("⚠️ FaceForensics++ 데이터가 없습니다!")
    print("   download_faceforensics.ipynb를 먼저 실행하세요")


---
## 4. 얼굴 추출 전처리


In [ ]:
# Dlib 얼굴 탐지기 초기화
detector = dlib.get_frontal_face_detector()

def extract_frames_from_video(video_path, num_frames=30):
    """비디오에서 균등하게 프레임 샘플링"""
    cap = cv2.VideoCapture(str(video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if total_frames == 0:
        cap.release()
        return []
    
    indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = []
    
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    cap.release()
    return frames

def detect_and_crop_face(image, margin=0.4, target_size=(224, 224)):
    """얼굴 탐지 및 크롭"""
    if isinstance(image, Image.Image):
        image = np.array(image)
    
    dets = detector(image, 1)
    
    if len(dets) == 0:
        return None
    
    d = max(dets, key=lambda x: (x.right() - x.left()) * (x.bottom() - x.top()))
    
    x1, y1, x2, y2 = d.left(), d.top(), d.right(), d.bottom()
    w, h = x2 - x1, y2 - y1
    
    x1 = max(0, int(x1 - w * margin))
    y1 = max(0, int(y1 - h * margin))
    x2 = min(image.shape[1], int(x2 + w * margin))
    y2 = min(image.shape[0], int(y2 + h * margin))
    
    face = image[y1:y2, x1:x2]
    face_pil = Image.fromarray(face).resize(target_size)
    
    return face_pil

print("✅ 얼굴 추출 함수 정의 완료")


In [ ]:
# 전처리 설정
NUM_FRAMES = 30
PROCESSED_DATA_DIR = f"{WORK_DIR}/processed_faces"
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(f"{PROCESSED_DATA_DIR}/real", exist_ok=True)
os.makedirs(f"{PROCESSED_DATA_DIR}/fake", exist_ok=True)

# FaceForensics++ 비디오 경로 설정
FF_DATA_PATH = "/content/drive/MyDrive/FaceForensics++"
VIDEO_DIRS = {
    'real': f"{FF_DATA_PATH}/original_sequences/youtube/c40/videos",
    'fake': f"{FF_DATA_PATH}/manipulated_sequences/Deepfakes/c40/videos"
}

print("📂 비디오 경로:")
print(f"  Real: {VIDEO_DIRS['real']}")
print(f"  Fake: {VIDEO_DIRS['fake']}")

# 비디오 파일 수집
video_files = {'real': [], 'fake': []}
for label, video_dir in VIDEO_DIRS.items():
    if os.path.exists(video_dir):
        files = list(Path(video_dir).glob('*.mp4'))
        video_files[label] = files
        print(f"\n✅ {label.upper()}: {len(files)} videos found")
    else:
        print(f"\n⚠️ {video_dir} not found!")
        print(f"   download_faceforensics.ipynb를 먼저 실행하세요")


In [ ]:
# 얼굴 추출 실행 (1~2시간 소요)
def preprocess_videos(video_files, label):
    """비디오에서 얼굴 추출 및 저장"""
    face_count = 0
    
    for video_path in tqdm(video_files, desc=f"Processing {label}"):
        video_name = video_path.stem
        
        frames = extract_frames_from_video(video_path, NUM_FRAMES)
        
        for i, frame in enumerate(frames):
            face = detect_and_crop_face(frame)
            if face is not None:
                save_path = f"{PROCESSED_DATA_DIR}/{label}/{video_name}_frame{i:03d}.jpg"
                face.save(save_path)
                face_count += 1
    
    return face_count

# Real/Fake 비디오 처리
for label in ['real', 'fake']:
    if len(video_files[label]) > 0:
        count = preprocess_videos(video_files[label], label)
        print(f"✅ {label.upper()} faces extracted: {count}")


---
## 5. 데이터셋 준비


In [ ]:
# 데이터 리스트 생성
real_faces = list(Path(f"{PROCESSED_DATA_DIR}/real").glob('*.jpg'))
fake_faces = list(Path(f"{PROCESSED_DATA_DIR}/fake").glob('*.jpg'))

print(f"Real faces: {len(real_faces)}")
print(f"Fake faces: {len(fake_faces)}")

data_list = []
for face_path in real_faces:
    data_list.append({'image_path': str(face_path), 'label': 0})

for face_path in fake_faces:
    data_list.append({'image_path': str(face_path), 'label': 1})

# Train/Val 분할 (80/20)
train_data, val_data = train_test_split(
    data_list, test_size=0.2, random_state=42, 
    stratify=[d['label'] for d in data_list]
)

print(f"\nTrain: {len(train_data)} (Real: {sum(1 for d in train_data if d['label']==0)}, Fake: {sum(1 for d in train_data if d['label']==1)})")
print(f"Val: {len(val_data)} (Real: {sum(1 for d in val_data if d['label']==0)}, Fake: {sum(1 for d in val_data if d['label']==1)})")


In [ ]:
# PyTorch Dataset
class DeepfakeDataset(Dataset):
    def __init__(self, data_list, processor):
        self.data_list = data_list
        self.processor = processor
    
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        item = self.data_list[idx]
        image = Image.open(item['image_path']).convert('RGB')
        
        inputs = self.processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze(0)
        
        return {
            'pixel_values': pixel_values,
            'labels': torch.tensor(item['label'], dtype=torch.long)
        }

print("✅ Dataset class 정의 완료")


---
## 6. 모델 로드 및 파인튜닝


In [ ]:
# 베이스라인 모델 로드 (Drive에서)
# 방법 1: Drive에 업로드한 경우
MODEL_PATH = "/content/drive/MyDrive/deepfake_models/deep-fake-detector-v2-model"

# 방법 2: 원본 HuggingFace 모델 사용 (인터넷 다운로드)
# MODEL_PATH = "google/vit-base-patch16-224-in21k"  # 베이스 모델

print(f"모델 로드 중: {MODEL_PATH}")

try:
    processor = ViTImageProcessor.from_pretrained(MODEL_PATH)
    model = ViTForImageClassification.from_pretrained(MODEL_PATH)
    print(f"✅ Model loaded from Drive!")
except:
    print("⚠️ Drive에 모델이 없습니다. 원본 ViT 사용...")
    # 베이스라인과 동일한 구조의 원본 모델
    MODEL_PATH = "google/vit-base-patch16-224-in21k"
    processor = ViTImageProcessor.from_pretrained(MODEL_PATH)
    model = ViTForImageClassification.from_pretrained(
        MODEL_PATH,
        num_labels=2,
        id2label={0: "Real", 1: "Deepfake"},
        label2id={"Real": 0, "Deepfake": 1}
    )
    print("✅ Model loaded from HuggingFace Hub (base model)")

print(f"   Parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

# Dataset 생성
train_dataset = DeepfakeDataset(train_data, processor)
val_dataset = DeepfakeDataset(val_data, processor)

print(f"✅ Train dataset: {len(train_dataset)}")
print(f"✅ Val dataset: {len(val_dataset)}")


In [ ]:
# Metric 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_fake = f1_score(labels, predictions, pos_label=1)
    
    return {
        'f1_macro': f1_macro,
        'f1_fake': f1_fake
    }

# Training Arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=True,
    dataloader_num_workers=2,
    remove_unused_columns=False,
    report_to="none",  # WandB 비활성화
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("✅ Trainer 초기화 완료")


In [ ]:
# 파인튜닝 시작 (2-3시간)
print("🚀 파인튜닝 시작...\n")
train_result = trainer.train()

print("\n✅ 파인튜닝 완료!")
print(f"   최종 Loss: {train_result.training_loss:.4f}")

# Validation 평가
eval_result = trainer.evaluate()

print("\n📊 Validation 결과:")
for key, value in eval_result.items():
    print(f"   {key}: {value:.4f}")


---
## 7. 모델 저장 및 다운로드


In [ ]:
# 로컬 저장
FINAL_MODEL_DIR = f"{OUTPUT_DIR}/final_model"
trainer.save_model(FINAL_MODEL_DIR)
processor.save_pretrained(FINAL_MODEL_DIR)

print(f"✅ 모델 저장: {FINAL_MODEL_DIR}")

# Drive 백업
import shutil
# FaceForensics++로 학습했지만 이름은 wilddeepfake_finetuned (이미 저장됨)
DRIVE_MODEL_PATH = f"{DRIVE_SAVE_DIR}/wilddeepfake_finetuned"
shutil.copytree(FINAL_MODEL_DIR, DRIVE_MODEL_PATH, dirs_exist_ok=True)

print(f"✅ Drive 백업: {DRIVE_MODEL_PATH}")
print(f"   (Note: 실제로는 FaceForensics++ 데이터로 학습됨)")


In [ ]:
# ZIP 압축 및 Drive 저장
!zip -r {OUTPUT_DIR}/wilddeepfake_model.zip {FINAL_MODEL_DIR}
!cp {OUTPUT_DIR}/wilddeepfake_model.zip {DRIVE_SAVE_DIR}/

print(f"✅ ZIP 저장: {DRIVE_SAVE_DIR}/wilddeepfake_model.zip")
print(f"   (Note: FaceForensics++ 파인튜닝 모델)")
print(f"   크기: ", end="")
!du -sh {DRIVE_SAVE_DIR}/wilddeepfake_model.zip


---
## 8. 다음 단계

### 로컬 제출 준비:
1. Drive에서 `wilddeepfake_model.zip` 다운로드
2. 압축 해제 → `submit/models/wilddeepfake_finetuned/` 배치
3. `submit/task.ipynb` 수정:
```python
MODEL_NAME = "./models/wilddeepfake_finetuned"
```
4. 제출:
```python
aif.submit(model_name="EXP-004-WildDeepfake", key="YOUR_KEY")
```

### 예상 성능:
- Baseline: 0.5489
- EXP-002: 0.5506 (+0.31%)
- EXP-003: 0.55~0.57 (TTA)
- **EXP-004: 0.60~0.65 (파인튜닝)**

Val F1이 0.70 이상이면 제출 강력 추천! 🚀
